Задание:
1.	Загрузить файл длиной не менее 2000 символов. 
2.	Составить программу, которая считает число уникальных слов в тексте (без критерия схожести)
3.	Составить программу, которая считает число гласных и согласных букв. 
4.	Составить программу, которая считает число предложений, их длину и число (количество) раз использования каждого слова в тексте (с критерием схожести, критерий схожести слов выбрать самостоятельно, например, spacy (en_core_web_sm) или расстояние Левенштейна). 
5.	Вывести 10 наиболее часто встречаемых слов. 

***
В данном блокноте выполняем пункт 4 и 5 задания.

***
Анализ по данным пунктам задания:
a) Для подсчета числа предложений можно находить знаки конца предложения (точку, вопросительный и восклицательный знак). Если между предыдущим таким знаком (или началом текста) и текущим знаком находится хотя бы один алфавитно-цифровой знак, то это и будет предложение.
Таким образом, можно разбить текст на список предложений. Длина списка будет числом предложений в тексте.
b) Для отыскания длины каждого предложения можно предварительно очистить каждое предложение в списке от знаков препинания, заменив их пробелами, и расщепить на список слов с помощью соответствующей процедуры Python. Длина этого списка будет длиной предложения.
c) Для нахождения числа раз использования каждого слова в тексте с использованием критерия схожести используем библиотеку spacy. 
Вначале мы уберем из текста все знаки препинания и разделим текст на отдельные слова. Получим список всех слов текста. Используя процедуру, отработанную на одном из предыдущих этапов (при выполнении второго пункта данного задания), уберем незначащие слова из списка.
Список будет включать все значащие слова,  причем столько раз, сколько имеется вхождений этого слова в исходный текст.
Простым преобразованием можно получить множество всех слов текста, где по определению каждое слово входит только одина раз.
Далее для каждого слова из множества, используя функции библиотеки spacy, можно найти в этом множестве список похожих на него слов. 
Эта процедура дает набор списков похожих слов, из которых состоит текст. Для каждого слова каждого списка можно найти количество его вхождений в очищенный текст. Взяв сумму числа вхождений для слов каждого списка, можно получить искомое число раз использования каждого слова в тексте.
Чтобы не было путаницы с регистрами текста, предварительно можно привести весь текст к нижнему регистру.

Тогда план работы по пункту 2 следующий.

1. Загрузить текст из файла.
2. Перевести текст в нижний регистр.
3. Проходить строку, включающую исследуемый текст, знак за знаком, находить знак конца предложения, то есть точку, восклицательный или положительный знак, и его положение в тексте. Для точки проверять, чтобы справа и слева не было цифр, иначе это будет десятичная точка числа, а не знак конца предложения.
4. Взяв слайс между предыдущим и текущим знаком конца предложения, проверить, есть ли в этом слайсе алфавитно-цифровые знаки. В случае наличия считаем полученный слайс очередным предложением текста. 
5. Далее из этого предложения создать новую строку, включающую только алфавитно-цифровые знаки, остальные знаки заменить пробелами. Затем расщепить полученную строку процедурой split и найти длину полученного списка. Это будет длина предложения, измеренная в словах. Пройдя таким образом все предложения текста, вывести  результат - список длин всех предложений - на экран.
6. Убрать в тексте, полученном в пункте 2, знаки препинания и шире - все неалфавитные знаки, заменив их пробелами.
7. Разбить текст на слова процедурой split, считая пробелы разделителями.
8. Из полученного списка слов убрать все стоп-слова. Получить тем самым список значащих слов текста.
9. Получить множество уникальных (пока без критерия схожести) слов текста, преобразовав список в множество.
10. Загрузить модель spacy en_core_web_sm. Для каждого слова множества значащих с помощью данной модели найти схожие с ним слова и число их вхождений в очищенный текст. Также занести в словарь номер каждой группы схожих слов и число упоминаний слов данной группы.
11. Отсортировать полученный словарь по значению ключа (дающего частотность соответствующей группы схожих слов)
12. Отобрать 10 наиболее часто встречающихся групп слов в полученном словаре.
14. Если в полученном словаре есть еще группы с той же частотностью, что и у десятой группы, также включить их в выводимый результат.

1. Загружаем текстовый файл:

In [22]:
import string, spacy

file_name = 'GospelJohn.txt' # file with text
# file_name = 'example.txt' # file with text
try:
    f = open(file_name,"r") # open file for reading
    text = f.read()         # reading file 
    f.close()               # closing file
    print('Read successfully')
except:
    print('Error reading file!')

Read successfully


2. Переводим текст в нижний регистр.

In [23]:
# to lower case:
text = text.lower()

3. Проходим строку, включающую исследуемый текст, знак за знаком, находим знак конца предложения, то есть точку, восклицательный или положительный знак, и его положение в тексте. Для точки проверяем, чтобы справа и слева не было цифр, иначе это будет десятичная точка числа, а не знак конца предложения.
4. Взяв слайс между предыдущим и текущим знаком конца предложения, проверяем, есть ли в этом слайсе алфавитно-цифровые знаки. В случае наличия считаем полученный слайс очередным предложением текста. 
5. Далее из этого предложения создаем новую строку, включающую только алфавитно-цифровые знаки, остальные знаки заменяем пробелами. Затем расщепляем полученную строку процедурой split и находим длину полученного списка. Это будет длина предложения, измеренная в словах. Пройдя таким образом все предложения текста, выводим  результат - список длин всех предложений - на экран.

In [24]:
# "left" - variable for left-hand end of a sentence
right = -1 # variable for right-hand end of a sentence
i = -1 # counter
sentences_length_list = list() # list for sentences lengths
for c in text:
    i += 1
    if c in ".?!":
        if c == '.' and text[i-1].isnumeric() and text[i+1].isnumeric():
            continue
        # new boundaries of a string:
        left = right + 1
        right = i
        astring = text[left:right + 1] # extract the string
        # print(f'String:\n{astring}')
        # if the string contains alphanumeric symbols, we suppose for it to be a sentence
        if len([ch for ch in astring if ch.isalnum()]) > 0:
            sentence_alphanumeric = "".join([ch if ch.isalnum else " " for ch in astring])
            sentence_length = len(sentence_alphanumeric.split()) # length of sentence in words
            sentences_length_list.append(sentence_length)
print(f'Lengths of sentences in the text (in words):\n {sentences_length_list}')
print(f'Number of sentences in the text:\n {len(sentences_length_list)}')
print(f'Average length of sentences in the text (in words):\n {round(sum(sentences_length_list) / len(sentences_length_list),1)}')

Lengths of sentences in the text (in words):
 [17, 8, 17, 12, 12, 11, 19, 14, 14, 18, 11, 47, 30, 29, 12, 15, 24, 22, 13, 6, 3, 6, 4, 4, 8, 11, 5, 24, 9, 23, 38, 12, 24, 22, 21, 20, 41, 13, 26, 11, 14, 15, 7, 20, 16, 22, 6, 25, 18, 11, 29, 14, 7, 19, 8, 21, 19, 20, 7, 30]
Number of sentences in the text:
 60
Average length of sentences in the text (in words):
 16.7


6. Убираем в тексте, полученном в пункте 2, знаки препинания и шире - все неалфавитные знаки, заменив их пробелами.

In [25]:
# replace non-alphanumeric signs by whitespaces:
text_no_signs = "".join([c if c.isalnum() else " " for c in text])

7. Разбиваем текст на слова процедурой split, считая пробелы разделителями.

In [26]:
word_list = text_no_signs.split() # split text into words and put to the list
print(word_list)

['in', 'the', 'beginning', 'was', 'the', 'word', 'and', 'the', 'word', 'was', 'with', 'god', 'and', 'the', 'word', 'was', 'god', 'the', 'same', 'was', 'in', 'the', 'beginning', 'with', 'god', 'all', 'things', 'were', 'made', 'by', 'him', 'and', 'without', 'him', 'was', 'not', 'any', 'thing', 'made', 'that', 'was', 'made', 'in', 'him', 'was', 'life', 'and', 'the', 'life', 'was', 'the', 'light', 'of', 'men', 'and', 'the', 'light', 'shineth', 'in', 'darkness', 'and', 'the', 'darkness', 'comprehended', 'it', 'not', 'there', 'was', 'a', 'man', 'sent', 'from', 'god', 'whose', 'name', 'was', 'john', 'the', 'same', 'came', 'for', 'a', 'witness', 'to', 'bear', 'witness', 'of', 'the', 'light', 'that', 'all', 'men', 'through', 'him', 'might', 'believe', 'he', 'was', 'not', 'that', 'light', 'but', 'was', 'sent', 'to', 'bear', 'witness', 'of', 'that', 'light', 'that', 'was', 'the', 'true', 'light', 'which', 'lighteth', 'every', 'man', 'that', 'cometh', 'into', 'the', 'world', 'he', 'was', 'in', 'th

8. Из полученного списка слов убираем все стоп-слова. Получаем тем самым список значащих слов текста.

Для этого надо вначале создать список стоп-слов
(источник: https://github.com/explosion/spaCy/blob/master/spacy/lang/en/stop_words.py)
Также мы немного расширяем этот список для учета устаревших вариантов слов, фигурирующих в нашем тексте

In [27]:
STOP_WORDS = set(
    """
a about above across after afterwards again against all almost alone along
already also although always am among amongst amount an and another any anyhow
anyone anything anyway anywhere are around as at

back be became because become becomes becoming been before beforehand behind
being below beside besides between beyond both bottom but by

call can cannot ca could

did do does doing done down due during

each eight either eleven else elsewhere empty enough even ever every
everyone everything everywhere except

few fifteen fifty first five for former formerly forty four from front full
further

get give go

had has have he hence her here hereafter hereby herein hereupon hers herself
him himself his how however hundred

i if in indeed into is it its itself

keep

last latter latterly least less

just

made make many may me meanwhile might mine more moreover most mostly move much
must my myself

name namely neither never nevertheless next nine no nobody none noone nor not
nothing now nowhere

of off often on once one only onto or other others otherwise our ours ourselves
out over own

part per perhaps please put

quite

rather re really regarding

s same say see seem seemed seeming seems serious several she should show side
since six sixty so some somehow someone something sometime sometimes somewhere
still such 

take ten than that the their them themselves then thence there thereafter
thereby therefore therein thereupon these they third this those though three
through throughout thru thus to together too top toward towards twelve twenty
two

under until up unless upon us used using

various very very via was we well were what whatever when whence whenever where
whereafter whereas whereby wherein whereupon wherever whether which while
whither who whoever whole whom whose why will with within without would

yet you your yours yourself yourselves
""".split()
)

contractions = ["n't", "nt", "'d", "d", "'ll", "ll", "'m", "m", "'re", "re", "'s", "s", "'ve", "ve"]
STOP_WORDS.update(contractions)

for apostrophe in ["‘", "’"]:
    for stopword in contractions:
        STOP_WORDS.add(stopword.replace("'", apostrophe))

obsoletisms = ["art", "hath", "shalt", "thou", "thee", "thyself", "ye", "ya", "unto"] # obsolete word forms
STOP_WORDS.update(obsoletisms)

print(STOP_WORDS)

{'hundred', 'everyone', 'alone', 'everywhere', 'myself', 'see', 'until', 'between', '’d', 'fifty', 'on', 'it', 'indeed', 'namely', 'very', 'most', 'becoming', 'so', 'becomes', 'are', 'besides', 'you', 'moreover', 'about', 'many', 'each', 'am', 'as', 'too', 'whose', 'who', 'me', 'while', 'put', 'top', 'just', 'in', 'their', 'whither', 'whereafter', 'being', 'art', 'and', 'throughout', "n't", 'keep', 'sometimes', 'really', 'he', 'two', 'empty', '’ll', 'third', 'ever', 'even', '‘re', 'first', 'three', 'mostly', 'few', 'unless', 'anyhow', 'thereafter', 'mine', 'noone', 'n‘t', 'beside', 'still', 'that', 'thereby', 'should', "'m", 'since', 'did', 'up', 'move', 'whatever', 'ya', 'never', 'her', 'where', 'one', '‘s', 'often', 'can', 'amongst', '’ve', 'us', 'yet', 'afterwards', '‘d', 'into', 'doing', "'d", 'else', 'around', 'however', 'meanwhile', 'none', 'whereupon', 'himself', 'others', 'became', 'before', 'nobody', 'because', '’re', '‘ll', 'five', 'call', 'its', 'they', 'back', 'full', 'ever

Продолжение п.8. 
Убираем из списка слова, если они входят в список стоп-слов:

In [28]:
word_list_new = list()
for aword in word_list:
    if aword not in STOP_WORDS:
        word_list_new.append(aword)

word_list = word_list_new
print(word_list)

['beginning', 'word', 'word', 'god', 'word', 'god', 'beginning', 'god', 'things', 'thing', 'life', 'life', 'light', 'men', 'light', 'shineth', 'darkness', 'darkness', 'comprehended', 'man', 'sent', 'god', 'john', 'came', 'witness', 'bear', 'witness', 'light', 'men', 'believe', 'light', 'sent', 'bear', 'witness', 'light', 'true', 'light', 'lighteth', 'man', 'cometh', 'world', 'world', 'world', 'world', 'knew', 'came', 'received', 'received', 'gave', 'power', 'sons', 'god', 'believe', 'born', 'blood', 'flesh', 'man', 'god', 'word', 'flesh', 'dwelt', 'beheld', 'glory', 'glory', 'begotten', 'father', 'grace', 'truth', 'john', 'bare', 'witness', 'cried', 'saying', 'spake', 'cometh', 'preferred', 'fulness', 'received', 'grace', 'grace', 'law', 'given', 'moses', 'grace', 'truth', 'came', 'jesus', 'christ', 'man', 'seen', 'god', 'time', 'begotten', 'son', 'bosom', 'father', 'declared', 'record', 'john', 'jews', 'sent', 'priests', 'levites', 'jerusalem', 'ask', 'confessed', 'denied', 'confessed

9. Получаем множество уникальных (пока без критерия схожести) слов текста, преобразовав список в множество.

In [29]:
word_set = set(word_list)
print(word_set)

{'wast', 'latchet', 'cried', 'bear', 'stood', 'word', 'city', 'dwellest', 'moses', 'tree', 'know', 'saith', 'confessed', 'philip', 'remaining', 'tenth', 'interpreted', 'fulness', 'grace', 'said', 'spake', 'time', 'followed', 'dove', 'angels', 'shoe', 'walked', 'gave', 'father', 'like', 'beheld', 'holy', 'answer', 'law', 'messias', 'nazareth', 'knowest', 'saying', 'baptize', 'galilee', 'follow', 'beginning', 'true', 'ascending', 'bosom', 'denied', 'stone', 'dwelt', 'lord', 'master', 'nathanael', 'knew', 'man', 'jesus', 'flesh', 'elias', 'abode', 'joseph', 'life', 'record', 'bare', 'glory', 'heaven', 'prophets', 'following', 'found', 'darkness', 'straight', 'power', 'cometh', 'esaias', 'baptizeth', 'christ', 'jews', 'day', 'declared', 'seen', 'worthy', 'brought', 'preferred', 'way', 'jordan', 'answered', 'shall', 'israel', 'sent', 'believe', 'voice', 'crying', 'bethabara', 'interpretation', 'king', 'guile', 'levites', 'baptizing', 'lamb', 'hour', 'behold', 'water', 'taketh', 'verily', 'b

10. Загружаем модель spacy en_core_web_sm. 
Для каждого слова множества значащих с помощью данной модели находим схожие с ним слова и число их вхождений в очищенный текст. 
Эти похожие слова заносим в список проверенных слов, чтобы на следующей итерации не считать их снова.
Повторяем для каждого следующего слова из множества значащих слов.
Попутно заносим в словарь номер каждой группы схожих слов и число упоминаний слов данной группы.
Выводим на экран группы схожих слов и число упоминаний слов этой группы в тексте.

In [34]:
# Loading the medium-sized spacy English model
nlp = spacy.load("en_core_web_md")
# Creating a SpaCy document. Since the argument has to be a string, 
# we have to make a string from the prepared word set first:
doc_spacy = nlp(" ".join([aword for aword in word_list]))  

word_groups = list()
current_word_set = set()
frequency_dict = dict()
words_checked = set()
i = 0 # word group counter
for aword in word_set:
    if aword not in words_checked:
        aword_nlp = nlp(aword) # 
        j = 0 # words in a group counter
        for w in doc_spacy:            
            if w.has_vector and aword_nlp.similarity(w) > 0.9:  # checking for similarity; 0.x is similarity threshold
                    print(f'Checking {aword_nlp.text} and {w.text}')
                    current_word_set.add(w.text) 
                    words_checked.add(w.text)           
                    j += 1
        word_groups.append(current_word_set) # appending the found set of similar words to global list
        frequency_dict[i] = j
        print(', '.join([w for w in current_word_set]))
        print(f'Number of these words in the text:\n {j}\n')
        current_word_set = set() # init the current set again
        i += 1 # increase counter
    
# print(f"Word set (to be empty): \n {word_set}")
# print(f"Word groups: \n {word_groups}")
# print(f'{frequency_dict}')
print(f"Word group quantity: \n {len(word_groups)}")

Checking wast and wast
wast
Number of these words in the text:
 1


Number of these words in the text:
 0

Checking cried and came
Checking cried and knew
Checking cried and came
Checking cried and gave
Checking cried and cried
Checking cried and came
Checking cried and asked
Checking cried and said
Checking cried and said
Checking cried and said
Checking cried and asked
Checking cried and said
Checking cried and said
Checking cried and knew
Checking cried and saw
Checking cried and knew
Checking cried and said
Checking cried and saw
Checking cried and stood
Checking cried and walked
Checking cried and heard
Checking cried and turned
Checking cried and saw
Checking cried and said
Checking cried and came
Checking cried and saw
Checking cried and heard
Checking cried and said
Checking cried and said
Checking cried and saw
Checking cried and said
Checking cried and saw
Checking cried and said
Checking cried and said
Checking cried and saw
gave, came, asked, cried, turned, stood, saw, knew

C:\Users\varaksa_yua\AppData\Local\Temp\ipykernel_8376\4060061897.py:17: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if w.has_vector and aword_nlp.similarity(w) > 0.9:  # checking for similarity; 0.x is similarity threshold


Checking ascending and descending
Checking ascending and descending
Checking ascending and ascending
Checking ascending and descending
descending, ascending
Number of these words in the text:
 4

Checking bosom and bosom
bosom
Number of these words in the text:
 1

Checking denied and denied
denied
Number of these words in the text:
 1

Checking stone and stone
stone
Number of these words in the text:
 1

Checking dwelt and dwelt
Checking dwelt and begotten
Checking dwelt and begotten
Checking dwelt and taketh
Checking dwelt and dwelt
dwelt, taketh, begotten
Number of these words in the text:
 5

Checking lord and darkness
Checking lord and darkness
Checking lord and lord
Checking lord and sin
Checking lord and ghost
Checking lord and king
king, ghost, sin, darkness, lord
Number of these words in the text:
 6

Checking master and master
master
Number of these words in the text:
 1

Checking nathanael and nathanael
Checking nathanael and nathanael
Checking nathanael and nathanael
Checki

11. Сортируем полученный словарь по значению ключа (дающего частотность соответствующей группы схожих слов)
12. Отбираем 10 наиболее часто встречающихся групп слов в полученном словаре.
13. Если в полученном словаре есть еще группы с той же частотностью, что и у десятой группы, также включаем их в выводимый результат. 

In [31]:
frequency_dict_sorted = sorted(frequency_dict.items(), key = lambda item: item[1])
print('The most frequent word groups:')

# extracting the most frequent words
if len(word_groups) > 10:
    max_num = 10
else:
    max_num = len(word_groups)

for i in range(max_num):
    group_num = frequency_dict_sorted[-i-1][0]
    word_group = word_groups[group_num]
    frequency = frequency_dict_sorted[-i-1][1]
    print(f'{i+1}. The word group\n"{word_group}"\n - has the frequency {frequency}')

# extracting also the 11th, 12th etc, if the frequency is the same as for the 10th one:
j = 1
while (i+j < len(word_groups)-2) and (frequency_dict_sorted[-i-j-1][1] == frequency_dict_sorted[-i-1][1]):
    group_num = frequency_dict_sorted[-i - j - 1][0]
    word_group = word_groups[group_num]
    frequency = frequency_dict_sorted[-i - j -1][1]
    print(f'{i + j + 1}. The word group\n"{word_group}"\n - has the frequency {frequency}')
    j += 1

The most frequent word groups:
1. The word group
"{'verily', 'saith', 'forth', 'prophets', 'holy', 'cometh', 'disciples', 'behold', 'fulness', 'spake', 'prophet'}"
 - has the frequency 11
2. The word group
"{'gave', 'came', 'asked', 'cried', 'turned', 'stood', 'saw', 'knew', 'heard', 'said', 'walked'}"
 - has the frequency 11
3. The word group
"{'good', 'called', 'thing', 'come', 'straight', 'things', 'way', 'know', 'time'}"
 - has the frequency 9
4. The word group
"{'joseph', 'simon', 'john', 'andrew', 'peter'}"
 - has the frequency 5
5. The word group
"{'king', 'ghost', 'sin', 'darkness', 'lord'}"
 - has the frequency 5
6. The word group
"{'christ', 'pharisees', 'jesus', 'priests'}"
 - has the frequency 4
7. The word group
"{'follow', 'speak', 'sent', 'seek'}"
 - has the frequency 4
8. The word group
"{'ask', 'write', 'answer', 'answered'}"
 - has the frequency 4
9. The word group
"{'world', 'life', 'spirit'}"
 - has the frequency 3
10. The word group
"{'man', 'sons', 'son'}"
 - has 

Итак, нами был проделан анализ англоязычного текста в отношении количества слов в предложениях, а также количества уникальных слов в тексте с объединением слов по критерию схожести.

Выводы:
1. В анализируемом классическом англоязычном тексте 17 в. тексте на ~1000 слов имеется 60 предложений, в среднем по 16.7 слов в каждом.
3. В данном тексте после удаления стоп-слов и объединения оставшихся по принципу схожести оказалось 99 групп слов. Таким образом, отдельных групп оказалось в десять раз меньше,  чем общее число всех слов.
4. Десять отобранных уникальных слов с максимальной частотностью дают хорошее представление о тематике текста.